In [1]:
import numpy as np
import scipy as sp
import pandas as pd
import timeit
import datetime
import time
import pprint
import itertools
import pickle
import sklearn
import dask
import dask.dataframe as dd
import dask.array as da
import os
os.chdir('/mnt/t48/bighomes-active/sfeng/patentdiffusion/')
import fastparquet
seed = 3
import seaborn as sns
import matplotlib
import matplotlib.pyplot as plt
# Distances
import scipy.spatial.distance as distance
# KL
from scipy.stats import entropy
# Normalize
from sklearn.preprocessing import normalize
from sklearn.preprocessing import StandardScaler
scaler = StandardScaler()
# Pairwise distances
from sklearn.metrics.pairwise import pairwise_distances
import h5py

/homedir/eco/sfeng/bigdata/python/miniconda3/lib/python3.6/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters


In [44]:
k = "primclass"
ns = fastparquet.ParquetFile("DataStore/2019-01/{0}_pr_cite_0110.parq".format(k)).to_pandas()
pdf = fastparquet.ParquetFile("RawData/Cleaned/patent_loc_unique_us_0628.parq").to_pandas(["patent", "gyear"])
ns["tp_gyear"] = ns["tp"].map(dict(zip(pdf["patent"], pdf["gyear"])))
ns["op_gyear"] = ns["op"].map(dict(zip(pdf["patent"], pdf["gyear"])))
ns["year_match"] = (ns["tp_gyear"] == ns["op_gyear"])
ns["inv_msa_match"] = (ns["tp_inv_msa"] == ns["op_inv_msa"])
del(pdf)

#Primclass
pdf = fastparquet.ParquetFile("RawData/Cleaned/patent_loc_unique_us_0628.parq").to_pandas(["patent", "primclass"])
ns["tp_primclass"] = ns["tp"].map(dict(zip(pdf["patent"], pdf["primclass"])))
ns["op_primclass"] = ns["op"].map(dict(zip(pdf["patent"], pdf["primclass"])))
ns["primclass_match"] = (ns["tp_primclass"] == ns["op_primclass"])
del(pdf)

def get_year_group_10(x):
    if x in range(1975,1985):
        yg = "1975-85"
    elif x in range(1985,1995):
        yg = "1985-95"
    elif x in range(1995, 2005):
        yg = "1995-05"
    elif x in range(2005,2015):
        yg = "2005-15"
    else:
        yg = np.nan
    return yg

ns["year_group"] = ns["tp_gyear"].map(get_year_group_10)



In [16]:
cols = ["inv_msa_match", "primclass_match", "common_pat_inv", "common_cited_match", "common_npc_match", "year_match",
        "lawyer_match", "examiner_match",]
mes = ["direct_cite"]
for m in mes:
    print(m)
    res = pd.DataFrame()
    for c in cols:
        r = ns[["year_group", m, c]].groupby(["year_group", c]).mean().rename(columns={m:c})
        res = pd.concat([res,r], axis=1)

    display(np.round(res*100,3))

direct_cite


inv_msa_match  primclass_match  common_pat_inv  \
1975-85 False          0.007            0.002           0.012   
        True           0.079            0.196          12.320   
1985-95 False          0.009            0.003           0.017   
        True           0.107            0.226          13.158   
1995-05 False          0.006            0.003           0.013   
        True           0.089            0.157          11.049   
2005-15 False          0.003            0.002           0.006   
        True           0.050            0.084           8.634   

               common_cited_match  common_npc_match  year_match  lawyer_match  \
1975-85 False               0.012             0.024       0.024         0.016   
        True               13.785            40.000         NaN         1.825   
1985-95 False               0.014             0.032       0.033         0.023   
        True               11.310            13.333         NaN         1.786   
1995-05 False               0.010             0.025       0.026         0.020   
        True                7.931            16.393         NaN         2.009   
2005-15 False               0.003             0.012       0.015         0.011   
        True                3.259            20.430         NaN         1.390   

               examiner_match  
1975-85 False           0.016  
        True            0.486  
1985-95 False           0.025  
        True            0.548  
1995-05 False           0.021  
        True            0.845  
2005-15 False           0.012  
        True            0.688

## Results tables for match columns with ratios
### 1. Primclass
#### Direct Cite

In [49]:
res = {}

In [50]:
k = "primclass"
ns = fastparquet.ParquetFile("DataStore/2019-01/{0}_pr_cite_0110.parq".format(k)).to_pandas()
pdf = fastparquet.ParquetFile("RawData/Cleaned/patent_loc_unique_us_0628.parq").to_pandas(["patent", "gyear"])
ns["tp_gyear"] = ns["tp"].map(dict(zip(pdf["patent"], pdf["gyear"])))
ns["op_gyear"] = ns["op"].map(dict(zip(pdf["patent"], pdf["gyear"])))
ns["year_match"] = (ns["tp_gyear"] == ns["op_gyear"])
ns["inv_msa_match"] = (ns["tp_inv_msa"] == ns["op_inv_msa"])
del(pdf)

#Primclass
pdf = fastparquet.ParquetFile("RawData/Cleaned/patent_loc_unique_us_0628.parq").to_pandas(["patent", "primclass"])
ns["tp_primclass"] = ns["tp"].map(dict(zip(pdf["patent"], pdf["primclass"])))
ns["op_primclass"] = ns["op"].map(dict(zip(pdf["patent"], pdf["primclass"])))
ns["primclass_match"] = (ns["tp_primclass"] == ns["op_primclass"])
del(pdf)

def get_year_group_10(x):
    if x in range(1975,1985):
        yg = "1975-85"
    elif x in range(1985,1995):
        yg = "1985-95"
    elif x in range(1995, 2005):
        yg = "1995-05"
    elif x in range(2005,2015):
        yg = "2005-15"
    else:
        yg = np.nan
    return yg

ns["year_group"] = ns["tp_gyear"].map(get_year_group_10)

ns = ns.loc[ns["year_match"] == False].copy()
print(len(ns))
ns = ns.dropna(subset=["inv_msa_match", "lawyer_match", "examiner_match"],how="any")
print(len(ns))

3617445
3617445


In [51]:
kn = "Primclass"
m = "direct_cite"
var_list = [("inv_msa_match", "MSA"), ("lawyer_match", "Lawyer"), ("examiner_match", "Examiner")]

for c,d in var_list:
    print(c,d)
    # Groupby common cited
    cc_g = ns.loc[ns[c] == True, [m, "year_group"]].groupby(["year_group"])
    cn_g = ns.loc[ns[c] == False, [m, "year_group"]].groupby(["year_group"])
    p_g = ns[[m, "year_group"]].groupby(["year_group"])

    # Mean
    cc_m = cc_g.mean().reset_index().rename(columns={"year_group": "Year Group", m: "{0} Match = T".format(d)})
    cn_m = cn_g.mean().reset_index()
    p_m = p_g.mean().reset_index()
    cc_m["{0} Match = F".format(d)] = cn_m[m]
    cc_m["Within-{0}".format(kn)] = p_m[m]
    cc_m["Ratio"] = cc_m["{0} Match = T".format(d)]/cc_m["{0} Match = F".format(d)]
    cc_m["{0} Match = F".format(d)] = cc_m["{0} Match = F".format(d)]*100
    cc_m["{0} Match = T".format(d)] = cc_m["{0} Match = T".format(d)]*100
    cc_m["Within-{0}".format(kn)] = cc_m["Within-{0}".format(kn)]*100
    # p-value
#     p = [sp.stats.ttest_ind(g[m], cn_g.get_group(n)[m], equal_var=False, nan_policy="omit")[1] for n,g in cc_g]
#     cc_m["$p$-value"] = p
    cc_m = np.round(cc_m.set_index("Year Group").T,3)
    res[(k,m,c)] = cc_m
    display(cc_m)
    print(cc_m.to_latex(escape=False, column_format="lcccc"))

inv_msa_match MSA


Year Group,1975-85,1985-95,1995-05,2005-15
MSA Match = T,0.456,0.663,0.427,0.206
MSA Match = F,0.087,0.104,0.072,0.033
Within-Primclass,0.155,0.211,0.148,0.073
Ratio,5.271,6.398,5.960,6.146


\begin{tabular}{lcccc}
\toprule
Year Group &  1975-85 &  1985-95 &  1995-05 &  2005-15 \\
\midrule
MSA Match = T    &    0.456 &    0.663 &    0.427 &    0.206 \\
MSA Match = F    &    0.087 &    0.104 &    0.072 &    0.033 \\
Within-Primclass &    0.155 &    0.211 &    0.148 &    0.073 \\
Ratio            &    5.271 &    6.398 &    5.960 &    6.146 \\
\bottomrule
\end{tabular}

lawyer_match Lawyer


Year Group,1975-85,1985-95,1995-05,2005-15
Lawyer Match = T,5.304,6.488,5.880,3.989
Lawyer Match = F,0.123,0.164,0.120,0.057
Within-Primclass,0.155,0.211,0.148,0.073
Ratio,42.992,39.640,48.935,70.545


\begin{tabular}{lcccc}
\toprule
Year Group &  1975-85 &  1985-95 &  1995-05 &  2005-15 \\
\midrule
Lawyer Match = T &    5.304 &    6.488 &    5.880 &    3.989 \\
Lawyer Match = F &    0.123 &    0.164 &    0.120 &    0.057 \\
Within-Primclass &    0.155 &    0.211 &    0.148 &    0.073 \\
Ratio            &   42.992 &   39.640 &   48.935 &   70.545 \\
\bottomrule
\end{tabular}

examiner_match Examiner


Year Group,1975-85,1985-95,1995-05,2005-15
Examiner Match = T,0.347,0.476,0.515,0.388
Examiner Match = F,0.122,0.176,0.126,0.060
Within-Primclass,0.155,0.211,0.148,0.073
Ratio,2.850,2.709,4.092,6.423


\begin{tabular}{lcccc}
\toprule
Year Group &  1975-85 &  1985-95 &  1995-05 &  2005-15 \\
\midrule
Examiner Match = T &    0.347 &    0.476 &    0.515 &    0.388 \\
Examiner Match = F &    0.122 &    0.176 &    0.126 &    0.060 \\
Within-Primclass   &    0.155 &    0.211 &    0.148 &    0.073 \\
Ratio              &    2.850 &    2.709 &    4.092 &    6.423 \\
\bottomrule
\end{tabular}



#### Similarity

In [59]:
ns = fastparquet.ParquetFile("DataStore/2018-11/primclass_sim_claims_1120.parq").to_pandas()
pdf = fastparquet.ParquetFile("RawData/Cleaned/patent_loc_unique_us_0628.parq").to_pandas(["patent", "gyear"])
ns["tp_gyear"] = ns["tp"].map(dict(zip(pdf["patent"], pdf["gyear"])))
ns["op_gyear"] = ns["op"].map(dict(zip(pdf["patent"], pdf["gyear"])))
ns["year_match"] = (ns["tp_gyear"] == ns["op_gyear"])
ns = ns.loc[ns["year_match"] == False]

In [60]:
kn = "Primclass"
m = "sim_docvecs"
var_list = [("inv_msa_match", "MSA"), ("lawyer_match", "Lawyer"), ("examiner_match", "Examiner")]

for c,d in var_list:
    print(c,d)
    # Groupby common cited
    cc_g = ns.loc[ns[c] == True, [m, "year_group"]].groupby(["year_group"])
    cn_g = ns.loc[ns[c] == False, [m, "year_group"]].groupby(["year_group"])
    p_g = ns[[m, "year_group"]].groupby(["year_group"])

    # Mean
    cc_m = cc_g.mean().reset_index().rename(columns={"year_group": "Year Group", m: "{0} Match = T".format(d)})
    cn_m = cn_g.mean().reset_index()
    p_m = p_g.mean().reset_index()
    cc_m["{0} Match = F".format(d)] = cn_m[m]
    cc_m["Within-{0}".format(kn)] = p_m[m]
    cc_m["Ratio"] = cc_m["{0} Match = T".format(d)]/cc_m["{0} Match = F".format(d)]
#     cc_m["{0} Match = F".format(d)] = cc_m["{0} Match = F".format(d)]*100
#     cc_m["{0} Match = T".format(d)] = cc_m["{0} Match = T".format(d)]*100
#     cc_m["Within-{0}".format(kn)] = cc_m["Within-{0}".format(kn)]*100
    # p-value
#     p = [sp.stats.ttest_ind(g[m], cn_g.get_group(n)[m], equal_var=False, nan_policy="omit")[1] for n,g in cc_g]
#     cc_m["$p$-value"] = p
    cc_m = np.round(cc_m.set_index("Year Group").T,3)
    res[(k,m,c)] = cc_m
    display(cc_m)
    print(cc_m.to_latex(escape=False, column_format="lcccc"))

inv_msa_match MSA


Year Group,1975-85,1985-95,1995-05,2005-15
MSA Match = T,0.196,0.193,0.195,0.197
MSA Match = F,0.190,0.182,0.184,0.188
Within-Primclass,0.191,0.184,0.186,0.190
Ratio,1.033,1.060,1.064,1.043


\begin{tabular}{lcccc}
\toprule
Year Group &  1975-85 &  1985-95 &  1995-05 &  2005-15 \\
\midrule
MSA Match = T    &    0.196 &    0.193 &    0.195 &    0.197 \\
MSA Match = F    &    0.190 &    0.182 &    0.184 &    0.188 \\
Within-Primclass &    0.191 &    0.184 &    0.186 &    0.190 \\
Ratio            &    1.033 &    1.060 &    1.064 &    1.043 \\
\bottomrule
\end{tabular}

lawyer_match Lawyer


Year Group,1975-85,1985-95,1995-05,2005-15
Lawyer Match = T,0.274,0.260,0.268,0.258
Lawyer Match = F,0.191,0.184,0.186,0.190
Within-Primclass,0.191,0.184,0.186,0.190
Ratio,1.440,1.418,1.442,1.356


\begin{tabular}{lcccc}
\toprule
Year Group &  1975-85 &  1985-95 &  1995-05 &  2005-15 \\
\midrule
Lawyer Match = T &    0.274 &    0.260 &    0.268 &    0.258 \\
Lawyer Match = F &    0.191 &    0.184 &    0.186 &    0.190 \\
Within-Primclass &    0.191 &    0.184 &    0.186 &    0.190 \\
Ratio            &    1.440 &    1.418 &    1.442 &    1.356 \\
\bottomrule
\end{tabular}

examiner_match Examiner


Year Group,1975-85,1985-95,1995-05,2005-15
Examiner Match = T,0.208,0.198,0.199,0.205
Examiner Match = F,0.188,0.182,0.185,0.190
Within-Primclass,0.191,0.184,0.186,0.190
Ratio,1.105,1.089,1.076,1.079


\begin{tabular}{lcccc}
\toprule
Year Group &  1975-85 &  1985-95 &  1995-05 &  2005-15 \\
\midrule
Examiner Match = T &    0.208 &    0.198 &    0.199 &    0.205 \\
Examiner Match = F &    0.188 &    0.182 &    0.185 &    0.190 \\
Within-Primclass   &    0.191 &    0.184 &    0.186 &    0.190 \\
Ratio              &    1.105 &    1.089 &    1.076 &    1.079 \\
\bottomrule
\end{tabular}



### Merged tables

In [61]:
res.keys()
for c in ["inv_msa_match", "lawyer_match", "examiner_match"]:
    print(c)
    tab = pd.concat([res[("primclass", "direct_cite", c)], res[("primclass", "sim_docvecs", c)]], axis=1)
    display(c)
    print(tab.to_latex(escape=False, column_format="lcccc|cccc"))

inv_msa_match


'inv_msa_match'

\begin{tabular}{lcccc|cccc}
\toprule
Year Group &  1975-85 &  1985-95 &  1995-05 &  2005-15 &  1975-85 &  1985-95 &  1995-05 &  2005-15 \\
\midrule
MSA Match = T    &    0.456 &    0.663 &    0.427 &    0.206 &    0.196 &    0.193 &    0.195 &    0.197 \\
MSA Match = F    &    0.087 &    0.104 &    0.072 &    0.033 &    0.190 &    0.182 &    0.184 &    0.188 \\
Within-Primclass &    0.155 &    0.211 &    0.148 &    0.073 &    0.191 &    0.184 &    0.186 &    0.190 \\
Ratio            &    5.271 &    6.398 &    5.960 &    6.146 &    1.033 &    1.060 &    1.064 &    1.043 \\
\bottomrule
\end{tabular}

lawyer_match


'lawyer_match'

\begin{tabular}{lcccc|cccc}
\toprule
Year Group &  1975-85 &  1985-95 &  1995-05 &  2005-15 &  1975-85 &  1985-95 &  1995-05 &  2005-15 \\
\midrule
Lawyer Match = T &    5.304 &    6.488 &    5.880 &    3.989 &    0.274 &    0.260 &    0.268 &    0.258 \\
Lawyer Match = F &    0.123 &    0.164 &    0.120 &    0.057 &    0.191 &    0.184 &    0.186 &    0.190 \\
Within-Primclass &    0.155 &    0.211 &    0.148 &    0.073 &    0.191 &    0.184 &    0.186 &    0.190 \\
Ratio            &   42.992 &   39.640 &   48.935 &   70.545 &    1.440 &    1.418 &    1.442 &    1.356 \\
\bottomrule
\end{tabular}

examiner_match


'examiner_match'

\begin{tabular}{lcccc|cccc}
\toprule
Year Group &  1975-85 &  1985-95 &  1995-05 &  2005-15 &  1975-85 &  1985-95 &  1995-05 &  2005-15 \\
\midrule
Examiner Match = T &    0.347 &    0.476 &    0.515 &    0.388 &    0.208 &    0.198 &    0.199 &    0.205 \\
Examiner Match = F &    0.122 &    0.176 &    0.126 &    0.060 &    0.188 &    0.182 &    0.185 &    0.190 \\
Within-Primclass   &    0.155 &    0.211 &    0.148 &    0.073 &    0.191 &    0.184 &    0.186 &    0.190 \\
Ratio              &    2.850 &    2.709 &    4.092 &    6.423 &    1.105 &    1.089 &    1.076 &    1.079 \\
\bottomrule
\end{tabular}



### 2. NAICS
#### Direct Cite

In [62]:
k = "naics_name"
ns = fastparquet.ParquetFile("DataStore/2019-01/{0}_pr_cite_0110.parq".format(k)).to_pandas()
pdf = fastparquet.ParquetFile("RawData/Cleaned/patent_loc_unique_us_0628.parq").to_pandas(["patent", "gyear"])
ns["tp_gyear"] = ns["tp"].map(dict(zip(pdf["patent"], pdf["gyear"])))
ns["op_gyear"] = ns["op"].map(dict(zip(pdf["patent"], pdf["gyear"])))
ns["year_match"] = (ns["tp_gyear"] == ns["op_gyear"])
ns["inv_msa_match"] = (ns["tp_inv_msa"] == ns["op_inv_msa"])
del(pdf)

#Primclass
pdf = fastparquet.ParquetFile("RawData/Cleaned/patent_loc_unique_us_0628.parq").to_pandas(["patent", "primclass"])
ns["tp_primclass"] = ns["tp"].map(dict(zip(pdf["patent"], pdf["primclass"])))
ns["op_primclass"] = ns["op"].map(dict(zip(pdf["patent"], pdf["primclass"])))
ns["primclass_match"] = (ns["tp_primclass"] == ns["op_primclass"])
del(pdf)

def get_year_group_10(x):
    if x in range(1975,1985):
        yg = "1975-85"
    elif x in range(1985,1995):
        yg = "1985-95"
    elif x in range(1995, 2005):
        yg = "1995-05"
    elif x in range(2005,2015):
        yg = "2005-15"
    else:
        yg = np.nan
    return yg

ns["year_group"] = ns["tp_gyear"].map(get_year_group_10)

ns = ns.loc[ns["year_match"] == False].copy()
print(len(ns))
ns = ns.dropna(subset=["inv_msa_match", "lawyer_match", "examiner_match"],how="any")
print(len(ns))

3956703
3956703


In [63]:
kn = "NAICS"
m = "direct_cite"
var_list = [("inv_msa_match", "MSA"), ("lawyer_match", "Lawyer"), ("examiner_match", "Examiner")]

for c,d in var_list:
    print(c,d)
    # Groupby common cited
    cc_g = ns.loc[ns[c] == True, [m, "year_group"]].groupby(["year_group"])
    cn_g = ns.loc[ns[c] == False, [m, "year_group"]].groupby(["year_group"])
    p_g = ns[[m, "year_group"]].groupby(["year_group"])

    # Mean
    cc_m = cc_g.mean().reset_index().rename(columns={"year_group": "Year Group", m: "{0} Match = T".format(d)})
    cn_m = cn_g.mean().reset_index()
    p_m = p_g.mean().reset_index()
    cc_m["{0} Match = F".format(d)] = cn_m[m]
    cc_m["Within-{0}".format(kn)] = p_m[m]
    cc_m["Ratio"] = cc_m["{0} Match = T".format(d)]/cc_m["{0} Match = F".format(d)]
    cc_m["{0} Match = F".format(d)] = cc_m["{0} Match = F".format(d)]*100
    cc_m["{0} Match = T".format(d)] = cc_m["{0} Match = T".format(d)]*100
    cc_m["Within-{0}".format(kn)] = cc_m["Within-{0}".format(kn)]*100
    # p-value
#     p = [sp.stats.ttest_ind(g[m], cn_g.get_group(n)[m], equal_var=False, nan_policy="omit")[1] for n,g in cc_g]
#     cc_m["$p$-value"] = p
    cc_m = np.round(cc_m.set_index("Year Group").T,3)
    res[(k,m,c)] = cc_m
    display(cc_m)
    print(cc_m.to_latex(escape=False, column_format="lcccc"))

inv_msa_match MSA


Year Group,1975-85,1985-95,1995-05,2005-15
MSA Match = T,0.079,0.107,0.089,0.050
MSA Match = F,0.007,0.009,0.006,0.003
Within-NAICS,0.024,0.033,0.026,0.015
Ratio,11.198,11.246,14.821,18.385


\begin{tabular}{lcccc}
\toprule
Year Group &  1975-85 &  1985-95 &  1995-05 &  2005-15 \\
\midrule
MSA Match = T &    0.079 &    0.107 &    0.089 &    0.050 \\
MSA Match = F &    0.007 &    0.009 &    0.006 &    0.003 \\
Within-NAICS  &    0.024 &    0.033 &    0.026 &    0.015 \\
Ratio         &   11.198 &   11.246 &   14.821 &   18.385 \\
\bottomrule
\end{tabular}

lawyer_match Lawyer


Year Group,1975-85,1985-95,1995-05,2005-15
Lawyer Match = T,1.825,1.786,2.009,1.390
Lawyer Match = F,0.016,0.023,0.020,0.011
Within-NAICS,0.024,0.033,0.026,0.015
Ratio,116.862,78.594,101.838,131.096


\begin{tabular}{lcccc}
\toprule
Year Group &  1975-85 &  1985-95 &  1995-05 &  2005-15 \\
\midrule
Lawyer Match = T &    1.825 &    1.786 &    2.009 &    1.390 \\
Lawyer Match = F &    0.016 &    0.023 &    0.020 &    0.011 \\
Within-NAICS     &    0.024 &    0.033 &    0.026 &    0.015 \\
Ratio            &  116.862 &   78.594 &  101.838 &  131.096 \\
\bottomrule
\end{tabular}

examiner_match Examiner


Year Group,1975-85,1985-95,1995-05,2005-15
Examiner Match = T,0.486,0.548,0.845,0.688
Examiner Match = F,0.016,0.025,0.021,0.012
Within-NAICS,0.024,0.033,0.026,0.015
Ratio,29.604,21.601,40.107,57.685


\begin{tabular}{lcccc}
\toprule
Year Group &  1975-85 &  1985-95 &  1995-05 &  2005-15 \\
\midrule
Examiner Match = T &    0.486 &    0.548 &    0.845 &    0.688 \\
Examiner Match = F &    0.016 &    0.025 &    0.021 &    0.012 \\
Within-NAICS       &    0.024 &    0.033 &    0.026 &    0.015 \\
Ratio              &   29.604 &   21.601 &   40.107 &   57.685 \\
\bottomrule
\end{tabular}



#### Similarity

In [64]:
ns = fastparquet.ParquetFile("DataStore/2018-11/naics_name_sim_claims_1120.parq").to_pandas()
pdf = fastparquet.ParquetFile("RawData/Cleaned/patent_loc_unique_us_0628.parq").to_pandas(["patent", "gyear"])
ns["tp_gyear"] = ns["tp"].map(dict(zip(pdf["patent"], pdf["gyear"])))
ns["op_gyear"] = ns["op"].map(dict(zip(pdf["patent"], pdf["gyear"])))
ns["year_match"] = (ns["tp_gyear"] == ns["op_gyear"])
ns = ns.loc[ns["year_match"] == False]

In [66]:
kn = "NAICS"
m = "sim_docvecs"
var_list = [("inv_msa_match", "MSA"), ("lawyer_match", "Lawyer"), ("examiner_match", "Examiner")]

for c,d in var_list:
    print(c,d)
    # Groupby common cited
    cc_g = ns.loc[ns[c] == True, [m, "year_group"]].groupby(["year_group"])
    cn_g = ns.loc[ns[c] == False, [m, "year_group"]].groupby(["year_group"])
    p_g = ns[[m, "year_group"]].groupby(["year_group"])

    # Mean
    cc_m = cc_g.mean().reset_index().rename(columns={"year_group": "Year Group", m: "{0} Match = T".format(d)})
    cn_m = cn_g.mean().reset_index()
    p_m = p_g.mean().reset_index()
    cc_m["{0} Match = F".format(d)] = cn_m[m]
    cc_m["Within-{0}".format(kn)] = p_m[m]
    cc_m["Ratio"] = cc_m["{0} Match = T".format(d)]/cc_m["{0} Match = F".format(d)]
#     cc_m["{0} Match = F".format(d)] = cc_m["{0} Match = F".format(d)]*100
#     cc_m["{0} Match = T".format(d)] = cc_m["{0} Match = T".format(d)]*100
#     cc_m["Within-{0}".format(kn)] = cc_m["Within-{0}".format(kn)]*100
    # p-value
#     p = [sp.stats.ttest_ind(g[m], cn_g.get_group(n)[m], equal_var=False, nan_policy="omit")[1] for n,g in cc_g]
#     cc_m["$p$-value"] = p
    cc_m = np.round(cc_m.set_index("Year Group").T,3)
    res[(k,m,c)] = cc_m
    display(cc_m)
    print(cc_m.to_latex(escape=False, column_format="lcccc"))

inv_msa_match MSA


Year Group,1975-85,1985-95,1995-05,2005-15
MSA Match = T,0.127,0.127,0.133,0.145
MSA Match = F,0.124,0.120,0.125,0.137
Within-NAICS,0.125,0.122,0.127,0.139
Ratio,1.019,1.060,1.059,1.057


\begin{tabular}{lcccc}
\toprule
Year Group &  1975-85 &  1985-95 &  1995-05 &  2005-15 \\
\midrule
MSA Match = T &    0.127 &    0.127 &    0.133 &    0.145 \\
MSA Match = F &    0.124 &    0.120 &    0.125 &    0.137 \\
Within-NAICS  &    0.125 &    0.122 &    0.127 &    0.139 \\
Ratio         &    1.019 &    1.060 &    1.059 &    1.057 \\
\bottomrule
\end{tabular}

lawyer_match Lawyer


Year Group,1975-85,1985-95,1995-05,2005-15
Lawyer Match = T,0.152,0.170,0.192,0.191
Lawyer Match = F,0.125,0.121,0.127,0.139
Within-NAICS,0.125,0.122,0.127,0.139
Ratio,1.219,1.400,1.514,1.376


\begin{tabular}{lcccc}
\toprule
Year Group &  1975-85 &  1985-95 &  1995-05 &  2005-15 \\
\midrule
Lawyer Match = T &    0.152 &    0.170 &    0.192 &    0.191 \\
Lawyer Match = F &    0.125 &    0.121 &    0.127 &    0.139 \\
Within-NAICS     &    0.125 &    0.122 &    0.127 &    0.139 \\
Ratio            &    1.219 &    1.400 &    1.514 &    1.376 \\
\bottomrule
\end{tabular}

examiner_match Examiner


Year Group,1975-85,1985-95,1995-05,2005-15
Examiner Match = T,0.201,0.202,0.201,0.205
Examiner Match = F,0.124,0.120,0.127,0.139
Within-NAICS,0.125,0.122,0.127,0.139
Ratio,1.623,1.679,1.582,1.477


\begin{tabular}{lcccc}
\toprule
Year Group &  1975-85 &  1985-95 &  1995-05 &  2005-15 \\
\midrule
Examiner Match = T &    0.201 &    0.202 &    0.201 &    0.205 \\
Examiner Match = F &    0.124 &    0.120 &    0.127 &    0.139 \\
Within-NAICS       &    0.125 &    0.122 &    0.127 &    0.139 \\
Ratio              &    1.623 &    1.679 &    1.582 &    1.477 \\
\bottomrule
\end{tabular}



In [67]:
res.keys()
for c in ["inv_msa_match", "lawyer_match", "examiner_match"]:
    print(c)
    tab = pd.concat([res[("naics_name", "direct_cite", c)], res[("naics_name", "sim_docvecs", c)]], axis=1)
    display(c)
    print(tab.to_latex(escape=False, column_format="lcccc|cccc"))

inv_msa_match


'inv_msa_match'

\begin{tabular}{lcccc|cccc}
\toprule
Year Group &  1975-85 &  1985-95 &  1995-05 &  2005-15 &  1975-85 &  1985-95 &  1995-05 &  2005-15 \\
\midrule
MSA Match = T &    0.079 &    0.107 &    0.089 &    0.050 &    0.127 &    0.127 &    0.133 &    0.145 \\
MSA Match = F &    0.007 &    0.009 &    0.006 &    0.003 &    0.124 &    0.120 &    0.125 &    0.137 \\
Within-NAICS  &    0.024 &    0.033 &    0.026 &    0.015 &    0.125 &    0.122 &    0.127 &    0.139 \\
Ratio         &   11.198 &   11.246 &   14.821 &   18.385 &    1.019 &    1.060 &    1.059 &    1.057 \\
\bottomrule
\end{tabular}

lawyer_match


'lawyer_match'

\begin{tabular}{lcccc|cccc}
\toprule
Year Group &  1975-85 &  1985-95 &  1995-05 &  2005-15 &  1975-85 &  1985-95 &  1995-05 &  2005-15 \\
\midrule
Lawyer Match = T &    1.825 &    1.786 &    2.009 &    1.390 &    0.152 &    0.170 &    0.192 &    0.191 \\
Lawyer Match = F &    0.016 &    0.023 &    0.020 &    0.011 &    0.125 &    0.121 &    0.127 &    0.139 \\
Within-NAICS     &    0.024 &    0.033 &    0.026 &    0.015 &    0.125 &    0.122 &    0.127 &    0.139 \\
Ratio            &  116.862 &   78.594 &  101.838 &  131.096 &    1.219 &    1.400 &    1.514 &    1.376 \\
\bottomrule
\end{tabular}

examiner_match


'examiner_match'

\begin{tabular}{lcccc|cccc}
\toprule
Year Group &  1975-85 &  1985-95 &  1995-05 &  2005-15 &  1975-85 &  1985-95 &  1995-05 &  2005-15 \\
\midrule
Examiner Match = T &    0.486 &    0.548 &    0.845 &    0.688 &    0.201 &    0.202 &    0.201 &    0.205 \\
Examiner Match = F &    0.016 &    0.025 &    0.021 &    0.012 &    0.124 &    0.120 &    0.127 &    0.139 \\
Within-NAICS       &    0.024 &    0.033 &    0.026 &    0.015 &    0.125 &    0.122 &    0.127 &    0.139 \\
Ratio              &   29.604 &   21.601 &   40.107 &   57.685 &    1.623 &    1.679 &    1.582 &    1.477 \\
\bottomrule
\end{tabular}

